In [2]:
import os
import numpy as np
from numpy.linalg import norm
import h5py
import pandas as pd

from astropy.cosmology import FlatLambdaCDM, z_at_value
import astropy.units as u
import astropy.cosmology.units as cu
import astropy.constants as const

from scipy.interpolate import interp1d
from scipy.integrate import cumulative_trapezoid

import matplotlib.pyplot as plt
import time

In [ ]:
#at some point I should just make a Simulation class / object to store simulation name, boxsize, binsize, attributes, cosmology, etc.

In [ ]:
sim = 'L35n270TNG'
boxsize = 205000
binsize = 500 #ckpc/h
nbins = int(boxsize/binsize)
outpath = f'n_e_maps/{args.sim}'

In [4]:
#when ray tracing through fine bins, we only need a (unordered) list of boxes (xyz) that the ray passes through
#when ray tracing through coarse bins, we need to access which coarse bins (bin index) that the ray passes through + where it crosses
#when ray tracing through sim boxes, we need to know which sim box (xyz) the the ray passes through + where it crosses

#returns ordered list of box crossings: coordinates where the ray enters that box, plus dest
#assumes that edges are located at integer multiples of boxsize

def get_box_crossings(dest, origin, boxsize): 
    
    edges_list = [dest]
    edge_dists_list = [norm(dest-origin)]
    
    for i in range(3): #loop over dimensions, get all edge crossings in each dimension
        x_f = dest[i]
        x_0 = origin[i]
        if (x_f//boxsize) == (x_0//boxsize): #no edge crossings
            continue
            
        unit_vec_i = (dest - origin)/(x_f - x_0)
        edge_1 = origin + unit_vec_i * (boxsize - x_0%boxsize)
        
        q = (x_f - edge_1[i]) // boxsize + 1 # no. of total edge crossings in the x-direction
        
        edges = edge_1 + np.atleast_2d(unit_vec_i) * boxsize * np.atleast_2d(np.arange(q)).T
        edges_list.append(edges)
        edge_dists_list.append(norm(edges - origin, axis=1))
        
    all_edges = np.unique(np.vstack(edges_list))
    all_dists = np.concatenate(edge_dists_list)

    all_dists, index_arr = np.unique(all_dists, return_index=True)
    all_edges = all_edges[index_arr]
    all_box_gridcoords = np.array(all_edges // boxsize, type=int)
    
    return all_edges, all_edge_dists, all_box_gridcoords
                 

In [ ]:
def ray_trace(dest, origin, boxsize, binsize, n_bins, snap_xs):
    
    x_edges = [0] #edge positions: for riemann integration
    xs = [] #bin midpoints, for calculating redshift, n_e(x), etc.
    nes = [] #electron density in (ckpc/h)**-3
    
    Vcell = binsize**3
    
    sim_box_edges, sim_box_edge_dists, sim_box_gridcoords = get_box_crossings(dest, origin, boxsize)
    current_pos = origin
    traveled_dist = 0
    
    open_snap = 99
    open_map = np.load(os.path.join(outpath, '99.npy'))
    
    for box_idx in range(len(n_sim_boxes)):
        
        offset = boxsize * sim_box_gridcoords #offset sim box coordinates
        next_pos = sim_box_edges[box_idx]
        
        current_pos -= offset
        next_pos -= offset
        
        bin_edges, bin_edge_dists, bin_gridcoords = get_box_crossings(next_pos, current_pos, binsize)
        bin_edge_dists += traveled_dist
        bin_mid_dists = np.convolve( np.concatenate([traveled_dist, bin_edge_dists]), [0.5, 0.5], 'valid' )
        
        bin_indices = bin_gridcoords[:, 0] * n_bins**2 + bin_gridcoords[:, 1] * n_bins + bin_gridcoords[:, 2]
        bin_snaps = (np.abs(np.repeat(bin_mid_dists, 100, axis=0).T - snap_xs)).argmin(axis=0) #find closest snapshot for each bin
        
        for bidx in range(len(bin_edges)-1): #loop through bins
            snap = bin_snaps[bidx]
            if snap != open_snap: #if corresponding snapshot is not open, open it
                open_snap = snap
                open_map = np.load(os.path.join(outpath, f'{snap}.npy'))
            
            x_edges.append(bin_edge_dists[bin_idx])
            xs.append(bin_mid_dists[bin_idx])
            nes.append(open_map[bin_indices[bidx]] / Vcell)
        
        current_pos = next_pos
        traveled_dist = sim_box_edge_dists[box_idx]
    
    return np.array(x_edges), np.array(xs), np.array(nes) 
    #returns n_e(x) where x is in ckpc/h, n_e is in (ckpc/h)**3, in order of increasing comoving distance (x)
    #x_edges is 1 longer than nes
        

In [ ]:
def compute_DM(x_edges, xs, nes, dist, cumulative=False):
    
    nes = nes.to(u.cm**(-3), h_equiv)
    zs = z_from_dist(xs)
    
    y = nes * (1 + zs) 
    dx = np.diff( (x_edges * u.kpc/cu.littleh).to(u.pc, h_equiv) )
    
    if cumulative:
        return np.cumsum(np.flip(y * dx)) #integrate from FRB to observer
        #plot with xs (bin midpoints)
    else:
        return np.sum(y * dx)

In [4]:
#set basepath and retrieve relevant cosmological parameters

# sim = 'L205n1250TNG'
sim = 'L35n270TNG'
basepath = f'/home/tnguser/sims.TNG/{sim}/output'
datapath = f'n_e_maps/{sim}'

with h5py.File(os.path.join(basepath, 'snapdir_099/snap_099.0.hdf5')) as f:
    header = dict(f['Header'].attrs)

boxsize = int(header['BoxSize'])
h = header['HubbleParam']
Omega0 = header['Omega0']

cosmo = FlatLambdaCDM(H0=100*h, Om0=Omega0)
h_equiv = cu.with_H0(cosmo.H0)

In [ ]:
# create interpolant for determining redshifts from comoving distances
dist_max = 1e7 # u.kpc / cu.littleh
dist_vals = np.linspace(1, dist_max, 10000)
redshift_vals = np.asarray([z_at_value(cosmo.comoving_distance, dist*h*u.kpc) for dist in dist_vals])
z_from_dist = interp1d(dist_vals, redshift_vals)

In [ ]:
snap_zs = []
for snap in range(100):
    with h5py.File(get_chunk_path(snap)) as f:
        zs.append(f['Header'].attrs['Redshift'])
snap_zs = np.array(snap_zs) * cu.redshift
snap_xs = cosmo.comoving_distance(zs).to(u.kpc) / cu.littleh

In [7]:
np.array([0,1,2]) == 0

array([ True, False, False])

In [ ]:
def dist_to_line(point, line, line_len): #shortest distance from vector from origin to 'point', to the line between the origin and 'line'
    return norm(np.cross(point, line), axis=1) / line_len

def dist_along_line(point, line, line_len):
    return np.dot(point, line) / line_len

def get_interp_data_from_snap(snap, vec_0, dist_0, vec_1, dist_1, start_time): #interpolates between snapshot snap and snap+1
    snap_df = pd.DataFrame()
    for chunk in range(snap_info['NumChunks'][snap]): #iterate through chunks
        print(f'{time.time()-start_time:<7.2f}:    Reading file {get_chunk_path(snap, chunk)}')
        chunk_df_0 = get_data_from_chunk(get_chunk_path(snap, chunk), vec_0, vec_1 - vec_0, dist_1 - dist_0) #corresponds to snapshot snap
        print(f'{time.time()-start_time:<7.2f}:    Reading file {get_chunk_path(snap+1, chunk)}')
        chunk_df_1 = get_data_from_chunk(get_chunk_path(snap+1, chunk), vec_0, vec_1 - vec_0, dist_1 - dist_0) #corresponds to snapshot snap+1
        #interpolate between chunk_df_0 and chunk_df_1, weighting by distance
        dists = (np.array(chunk_df_1['Comoving Distance']) + np.array(chunk_df_0['Comoving Distance'])) / 2 #we will weight by the average comoving dist over these two snapshots
        interp_df = (chunk_df_1 - chunk_df_0)/(dist_1 - dist_0)*(dists - dist_0) + chunk_df_0
        interp_df.dropna(inplace=True)
        snap_df = pd.concat((snap_df, interp_df))
    return snap_df
    
def get_los_with_interp(dest, origin, radius=200, basepath=basepath, save_hdf=None):
    
    dest = np.array(dest)
    origin = np.array(origin)
    
    vec_0 = np.array(origin)
    dist_0 = norm(vec_0)
    total_vec = dest - origin
    total_dist = norm(total_vec)
    unit_vec = total_vec / total_dist
    
    res = pd.DataFrame()
    #ray trace through snapshots
    start_time = time.time()
    snap = 99
    
    while dist_0 < total_dist: 
        
        print(f'{time.time()-start_time:<7.2f}: Snapshot {snap}')
        
        dist_1 = min((snap_info['Comoving Distance'][snap-1], total_dist))
        vec_1 = origin + dist_1 * unit_vec
        print(f'Going from {vec_0} to {vec_1}')
    
        res = pd.concat(res, get_interp_data_from_snap(snap, vec_0, dist_0, vec_1, dist_1, start_time)) #interpolated between snapshots snap and snap+1
        
        snap -= 1
        dist_0, vec_0 = dist_1, vec_1
    
    res = res[~res.index.duplicated(keep='last')]
    header = pd.Series({'basepath': basepath, 'dest': dest, 'origin': origin, 'radius': radius, 'params': params})
    if save_hdf:
        res.to_hdf(save_hdf, key='data')
        header.to_hdf(save_hdf, key='header')
    
    print(f'Total Time: {time.time()-start_time:<7.2f}')
    return res, header

In [ ]:
zs = []
numchunks = []
for i in range(99, -1, -1):
    with h5py.File(il.snapshot.snapPath(basepath, i)) as f:
        zs.append(f['Header'].attrs['Redshift'])
        numchunks.append(f['Header'].attrs['NumFilesPerSnapshot'])
zs = np.array(zs) * cu.redshift
xs = cosmo.comoving_distance(zs).to(u.kpc) / cu.littleh
half_dists = np.convolve(xs, np.array([0.5, 0.5]))[1:] #cutoff at halfway comoving distance between different snapshot redshifts
half_dists[-1] = xs[-1]

snap_info = pd.DataFrame(data = {'NumChunks': numchunks, 'Redshift': zs, 'Comoving Distance': xs, 'Halfway Distances': dist_cutoffs}, 
                         index=range(99, -1, -1))
snap_info